# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Подготовка данных

### Инициализация сессии

В этом разделе мы импортируем все необходимые модули для трансформации признаков, алгоритм линейной регрессии, инициализируем Spark-сессию и откроем датасет.

In [1]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql import DataFrameStatFunctions as statFunc

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression

In [2]:
pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncoderEstimator

In [3]:
RANDOM_SEED = 2023

In [4]:
spark = SparkSession.builder.master("local").appName("House_cost_prediction").getOrCreate()

В колонках датасета содержатся следующие данные:
* 		longitude — широта;
* 		latitude — долгота;
* 		housing_median_age — медианный возраст жителей жилого массива;
* 		total_rooms — общее количество комнат в домах жилого массива;
* 		total_bedrooms — общее количество спален в домах жилого массива;
* 		population — количество человек, которые проживают в жилом массиве;
* 		households — количество домовладений в жилом массиве;
* 		median_income — медианный доход жителей жилого массива;
* 		median_house_value — медианная стоимость дома в жилом массиве;
* 		ocean_proximity — близость к океану.

In [5]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema=True)
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



### Подготовка данных

В колонке total_bedrooms есть небольшое количество пропусков. Не придумал, как их заменить на медиану, например, поэтому просто удалил)

In [6]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [11]:
median = statFunc(df).approxQuantile('total_bedrooms', [0.5], 0.25)
median

[296.0]

In [13]:
df = df.fillna(296, subset='total_bedrooms')

In [51]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,535.4448158914729,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.95872850464224,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Разделим наш датасет на категориальные, числовые колонки и выделим целевой признак.

In [52]:
categorial_col = ['ocean_proximity']
numerical_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 
                 'population', 'households', 'median_income']
target = 'median_house_value'

Далее трансформируем категориальный признак и создадим для него OHE.

In [53]:
indexer = StringIndexer(inputCols=categorial_col, outputCols=[c+'_idx' for c in categorial_col])
df = indexer.fit(df).transform(df)
cols = [c for c in df.columns for i in categorial_col if (c.startswith(i))]
df.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



In [54]:
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorial_col],
                       outputCols=[c+'_ohe' for c in categorial_col])
df = encoder.fit(df).transform(df)
cols = [c for c in df.columns for i in categorial_col if (c.startswith(i))]
df.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Объединим признаки в один вектор.

In [55]:
categorial_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in categorial_col],
                                      outputCol='categorial_features')
df = categorial_assembler.transform(df)

Трансформируем числовые признаки.

In [56]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                     outputCol='numerical_features')
df = numerical_assembler.transform(df)

In [57]:
standardScaler = StandardScaler(inputCol='numerical_features',
                               outputCol='numerical_features_scaled')
df = standardScaler.fit(df).transform(df)

После преобразований получился датасет с такими колонками.

In [58]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorial_features', 'numerical_features', 'numerical_features_scaled']


Соберем трансформированные категориальные и числовые признаки с помощью VectorAssembler.

In [59]:
all_features = ['categorial_features', 'numerical_features_scaled']

In [60]:
final_assembler = VectorAssembler(inputCols=all_features,
                                 outputCol='features')
df_all = final_assembler.transform(df)
df_all.select(all_features).show(3)

+-------------------+-------------------------+
|categorial_features|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[3],[1.0])|     [-61.007269596069...|
|      (4,[3],[1.0])|     [-61.002278409814...|
|      (4,[3],[1.0])|     [-61.012260782324...|
+-------------------+-------------------------+
only showing top 3 rows



## Обучение моделей

### Обучение на всех признаках

На данном этапе мы обучаем модель, используя все признаки.

Разделим датасет на обучающую и тестовую выборки в соотношении 75/25.

In [61]:
train_data, test_data = df_all.randomSplit([.75,.25], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

15477 5163


Для обучения используем модель линейной регрессии.

In [62]:
lr_all = LinearRegression(featuresCol='features', labelCol=target, maxIter=10, regParam=0.3, elasticNetParam=0.8)
model_all = lr_all.fit(train_data)

23/05/16 08:07:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/16 08:07:47 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Сделаем предсказания и выведем их в таблице.

In [63]:
lr_all_predictions = model_all.transform(test_data)
lr_all_predictions.select('prediction', target).show(5)

+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
| 155407.0639860659|           78300.0|
|169001.40441553912|           67000.0|
| 168349.9594074617|           81300.0|
| 185439.1338927421|           62500.0|
| 207749.6534203057|          100600.0|
+------------------+------------------+
only showing top 5 rows



Оценим качество модели с помощью заданных метрик.

In [64]:
trainingSummary = model_all.summary
print('RMSE: %f' % trainingSummary.rootMeanSquaredError)
print('MAE %f' % trainingSummary.meanAbsoluteError)
print('R2: %f' % trainingSummary.r2)

RMSE: 69131.318138
MAE 50197.727893
R2: 0.640946


### Обучение на числовых признаках

Отдельно обучим модель только на числовых признаках.

In [65]:
df_numerical = df.select(['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
                          'total_bedrooms', 'population', 'households', 'median_income', 
                          'median_house_value', 'numerical_features', 
                          'numerical_features_scaled'])

In [66]:
train_data, test_data = df_numerical.randomSplit([.75,.25], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

15477 5163


In [67]:
lr_numerical = LinearRegression(featuresCol='numerical_features_scaled', 
                      labelCol=target, maxIter=10, regParam=0.3, elasticNetParam=0.8)
model_numerical = lr_numerical.fit(train_data)

In [68]:
lr_num_predictions = model_numerical.transform(test_data)
lr_num_predictions.select('prediction', target).show(5)

+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
| 84562.60377605818|           78300.0|
|120767.50323410379|           67000.0|
|119327.50704700593|           81300.0|
| 137651.7899244805|           62500.0|
|159299.07395881647|          100600.0|
+------------------+------------------+
only showing top 5 rows



In [69]:
trainingSummary = model_numerical.summary
print('RMSE: %f' % trainingSummary.rootMeanSquaredError)
print('MAE %f' % trainingSummary.meanAbsoluteError)
print('R2: %f' % trainingSummary.r2)

RMSE: 69651.235250
MAE 51025.298537
R2: 0.635525


## Анализ результатов

В рамках задания мы обучили модель линейной регрессии на данных о жилье в Калифорнии в 1990 г. Нашей задачей было предсказать медианную стоимость жилья в жилом массиве. Для обучения мы использовали два набора данных - со всеми признаками и только с числовыми признаками. На числовых признаках результат по метрике R2 получился чуть лучше - 0.637699.